In [7]:
from spc_imports import *
set_up_plt()
import MDAnalysis as mda
import data_process
import glob

/Users/sperez/bin/PYTHON_SCRIPTS/spc_imports.py:21: MatplotlibDeprecationWarning: Support for setting the 'text.latex.preamble' or 'pgf.preamble' rcParam to a list of strings is deprecated since 3.3 and will be removed two minor releases later; set it to a single string instead.
  plt.rcParams["text.latex.preamble"] = [


In [ ]:
%%time
skip=10
div=skip*2
for treant in b:
    atoms=['N','C','CA','CB']
    iterables=[np.arange(26,122),atoms]
    indices=pd.MultiIndex.from_product(iterables, names=['resid','atom'])
    print(treant)
    top= md.load(treant['traj_comp_a_pbc_protein.xtc'].abspath,top=treant['topo_protein.psf'].abspath)
    shifts=[]
    shifts2=[]
    for chain in ["A","B","C","D"]:
        trj= md.load(treant['traj_comp_a_pbc_protein.xtc'].abspath,top=treant['topo_protein.psf'].abspath
                     ,atom_indices=top.topology.select(f"segname PRO{chain} and not element H and not name H HA"))
        shifts0=[]
        n_frames=trj.n_frames
        step=n_frames//div
        for j in tqdm(range(0,n_frames-step,step), desc=f't({chain})',leave=True,smoothing=1):
            shifts0.append(md.nmr.chemical_shifts_spartaplus(trj[j:j+step:skip]))
#         if n_frames%div >skip :
#             shifts0.append(md.nmr.chemical_shifts_spartaplus(trj[j+step:n_frames:skip]))
        shifts0=pd.concat(shifts0,axis=1)
        shifts0=shifts0.reindex(indices)
        shifts0.columns=np.arange(0,len(shifts0.columns))
        shifts.append(pd.DataFrame({'theo':shifts0.mean(axis=1)}))
        shifts2.append(shifts0)
    df=pd.concat(shifts)
    df=pd.DataFrame(np.hstack([df.groupby(['resid','atom']).mean(),
                       df.groupby(['resid','atom']).sem()]),
             columns=['mean','sem'],
            index=df.groupby(['resid','atom']).mean().index)
    df2=pd.concat(shifts2)
    df2.to_pickle(treant['NMR/sparta_plus_raw.pkl'].abspath)
    my_list0=[]
    for atom in atoms:
        my_list0.append(df['mean'].loc[:,atom].to_numpy())
    for atom in atoms:
        my_list0.append(df['sem'].loc[:,atom].to_numpy())
    iterables=[['mean','sem'],atoms]
    columns=pd.MultiIndex.from_product(iterables, names=['var','atom'])
    df=pd.DataFrame(np.vstack(my_list0).T,index=df.index.levels[0],columns=columns)
    df.to_pickle(treant['NMR/sparta_plus.pkl'].abspath)

In [12]:
raw_data_dir = '../data/raw/'

Load the trajectory

In [42]:
state = '5VKH_3FB5_string'
pdbs = sorted(glob.glob(raw_data_dir+f'{state}/*gro'))
univ = mda.Universe(raw_data_dir+f'{state}/topo.psf',pdbs)

/Users/sperez/bin/anaconda3/envs/NMR_assign_state/lib/python3.8/site-packages/MDAnalysis/coordinates/base.py:865: UserWarning: Reader has no dt information, set to 1.0 ps
  warnings.warn("Reader has no dt information, set to 1.0 ps")


Get the NMR shifts with ppm

In [66]:
def get_sparta_plus_chemical_shifts(univ,temp_dir='./'):
    import MDAnalysis as mda
    import mdtraj as md
    '''
    Use sparta_plus with the interface of mdtraj to get the chemical shifts of the trajectory.
    '''
    sel_protein=univ.select_atoms('protein')
    xtc = f"{temp_dir}/tmp.xtc"
    pdb = f"{temp_dir}/tmp.pdb"
    
    for segid in np.unique(sel_protein.segids):
        sel_segid=sel_protein.select_atoms(f'segid {segid} and not name H HA')    
        with mda.Writer(xtc, n_atoms=sel_segid.atoms.n_atoms) as W:
            for ts in univ.trajectory:
                W.write(sel_segid)
        sel_segid.write(pdb)
        
        #Load data and calculate NMR-CS
        trj= md.load(xtc,pdb)
        md.nmr.chemical_shifts_spartaplus(trj)
            
    os.remove(xtc)
    os.remove(pdb)   
    return df

In [67]:
get_sparta_plus_chemical_shifts(univ,'../data/interim/')

In [39]:
u.select_atoms('protein').segids

array(['PROA', 'PROA', 'PROA', ..., 'PROC', 'PROC', 'PROC'], dtype=object)